# Caminho da base de dados

In [0]:
path_lista_cidades = '/content/drive/My Drive/Datasets/GPS/lista_cidades.txt'
path_lista_distancia_cidades = '/content/drive/My Drive/Datasets/GPS/distancia_cidades.txt'
path_latitude_longitude = '/content/drive/My Drive/Datasets/GPS/latitude_longitude.txt'

# Classes do programa

## Classe do Mapa

In [0]:
class Mapa: 

  def __init__(self):
    self.cidades = {}
  
  def carregarCidades(self, caminho):
    with open(caminho, 'r') as arquivo: 
      lista_cidades = arquivo.readlines()
    
    for cid in lista_cidades: 
      c = cid.strip()
      self.cidades[c] = []
  
  def carregarCidadesVizinhas(self, caminho):
    with open(caminho,'r') as arquivo:
      vizinhanca = arquivo.readlines()
    
    for vizinho in vizinhanca: 
      linha_limpa = vizinho.strip()
      cid1, cid2, dist = linha_limpa.split(';')
      self.cidades[cid1].append((cid2,float(dist)))

  def gerar_sucessores(self,cidade):
      return self.cidades[cidade]
    


## Classes da Busca

In [148]:
class Estado:

  def __init__(self, identificador, custo, pai):
    self.identificador = identificador 
    self.custo = custo
    self.pai = pai
    self.heuristica 0
    self.avaliacao = 0


  def __repr__(self): 
    str_id = 'Identificador: ' + self.identificador
    str_pai = 'Pai: --'
    if self.pai != None: 
      str_pai = 'Pai: ' + self.pai
    str_custo = 'Custo: ' + str(self.custo)
    str_heuristica = 'Heuristica: ' + str(self.heuristica)
    str_avaliacao = 'Avaliação: ' + str(self.avaliacao)

    return str_id + ' | ' + str_pai + ' | ' + str_custo + ' | ' + ' | ' + str_heuristica + ' | ' + str_avaliacao 


SyntaxError: ignored

In [0]:
class Busca: 

  def __init__(self,estado_inicial, estado_objetivo, regra, estrategia, heuristica=None): 
    self.fronteira = []
    self.explorados = {}
    self.estado_objetivo = estado_objetivo 
    self.fronteira.append(estado_inicial)
    self.regra = regra 
    self.heuristica = heuristica
    self.estrategia = estrategia
    

  def buscar_solucao(self): 
    while len(self.fronteira) > 0:
      estado, self.fronteira = self.tirar_fronteira()
      print('Estado retirado da fronteira: ', estado)
      if estado.identificador == self.estado_objetivo.identificador:
        print('É o estado objetivo')
        print('Lista de explorados!!!')
        print(self.explorados)
        return self.retornar_caminho(estado)
      else: 
        print('Não é o objetivo')
        if estado.identificador in self.explorados: 
          if self.explorados[estado.identificador].custo > estado.custo:
            self.explorados[estado.identificador] = estado 
        else: 
          self.explorados[estado.identificador] = estado 
        adjacentes = self.regra.gerar_sucessores(estado.identificador)
        sucessores = self.gerar_sucessores(adjacentes, estado)
        self.fronteira = self.inserir_ordenado(self.fronteira, sucessores)
        print('Fronteira: ', self.fronteira)
    return None   
  
  def inserir_ordenado(self,fronteira,sucessores): 
    for sucessor in sucessores: 
      cont = 0;
      while cont < len(fronteira):
        if sucessor.avaliacao > fronteira[cont].avaliacao:
          cont = cont + 1
        else: 
          break 
      if cont < len(fronteira): 
        fronteira.insert(cont,sucessor)
      else:
        fronteira.append(sucessor)
    return fronteira


  def gerar_sucessores(self, adjacentes, estado_pai):
    lista_sucessores = [] 
    for adjacente in adjacentes:
      vizinho, custo_vizinho = adjacente
      estado = Estado(vizinho,estado_pai.custo + custo_vizinho,estado_pai.identificador)
      if self.heuristica != None: 
        self.heuristica.calc_heuristica(estado,self.estado_objetivo)
      self.estrategia.funcao_avaliacao(estado)
      if (adjacente not in self.explorados):         
        lista_sucessores.append(estado)
      elif self.explorados[estado.identificador].custo > estado.custo:
        lista_sucessores.append(estado)
      
    return lista_sucessores

  def tirar_fronteira(self):
    estado = self.fronteira[0]
    return estado, self.fronteira[1:]
  
  def retornar_caminho(self, estado): 
    print('========================')
    print('===RESULTADO DA BUSCA===')
    print('========================')
    print('Custo: ', estado.custo)
    
    caminho = []
    estado_atual = estado
    while(estado_atual.pai != None):
      caminho.append(estado_atual.identificador)
      estado_atual = self.explorados[estado_atual.pai]
    
    caminho.append(estado_atual.identificador)
    caminho.reverse()
    return caminho



In [0]:
class HeuristicaLinhaReta: 

  def __init__(self,caminho):
    self.coordenadas = {}
    with open(caminho,'r') as arquivo:
      for linha in arquivo: 
        cidade, latitude, longitude = linha.strip().split(';')
        self.coordenadas[cidade] = (float(latitude),float(longitude))

  def calc_heuristica(self, estado, estadoObj):
    coordenadas1 = self.coordenadas[estado.identificador]
    coordenadas2 = self.coordenadas[estadoObj.identificador]
    dist = ( (coordenadas1[0] - coordenadas2[0])**2 +  (coordenadas1[1] - coordenadas2[1])**2) ** 0.5
    estado.heuristica = dist


In [0]:
class BranchAndBound: 
  def funcao_avaliacao(self, estado): 
    estado.avaliacao = estado.custo

In [0]:
class BestFirst: 
  def funcao_avaliacao(self, estado):
    estado.avaliacao = estado.heuristica
    

In [0]:
class AStar:
  def funcao_avaliacao(self, estado): 
    estado.avaliacao = estado.custo + estado.heuristica

# Carregar Mapa

In [0]:
mapa = Mapa()
mapa.carregarCidades(path_lista_cidades)
mapa.carregarCidadesVizinhas(path_lista_distancia_cidades)

# Resto do Programa:


In [0]:
branch_and_bound = BranchAndBound()
best_first = BestFirst()
a_star = AStar()

In [0]:
estado_inicial = Estado('Arad',0,None,None)
estado_final = Estado('Bucharest',0,None,None)
heuristica = HeuristicaLinhaReta(path_latitude_longitude)
busca = Busca(estado_inicial, estado_final, mapa, a_star, heuristica)

In [159]:
busca.buscar_solucao()

Estado retirado da fronteira:  Identificador: Arad | Pai: -- | Custo: 0 |  | Heuristica: 0 | Avaliação: 0
Não é o objetivo
Fronteira:  [Identificador: Sibiu | Pai: Arad | Custo: 140.0 |  | Heuristica: 301.0813843464919 | Avaliação: 441.0813843464919, Identificador: Timisoara | Pai: Arad | Custo: 118.0 |  | Heuristica: 349.6069221282668 | Avaliação: 467.6069221282668, Identificador: Zerind | Pai: Arad | Custo: 75.0 |  | Heuristica: 397.0201506221063 | Avaliação: 472.0201506221063]
Estado retirado da fronteira:  Identificador: Sibiu | Pai: Arad | Custo: 140.0 |  | Heuristica: 301.0813843464919 | Avaliação: 441.0813843464919
Não é o objetivo
Fronteira:  [Identificador: Rimnicu Vilcea | Pai: Sibiu | Custo: 220.0 |  | Heuristica: 194.74342094150447 | Avaliação: 414.7434209415045, Identificador: Fagaras | Pai: Sibiu | Custo: 239.0 |  | Heuristica: 176.13914953808538 | Avaliação: 415.13914953808535, Identificador: Timisoara | Pai: Arad | Custo: 118.0 |  | Heuristica: 349.6069221282668 | Avali

['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest']

# Área de testes

In [0]:
lista = [5,10,20]

In [144]:
lista

[5, 10, 20]

In [0]:
lista.insert(2,15)

In [146]:
lista

[5, 10, 15, 20]